# Exploratory Data Analysis — Kaggle House Prices (Advanced Regression Techniques)

This notebook explores the Kaggle House Prices competition dataset. It documents columns, distributions, missingness, feature engineering ideas, and baseline modeling.

- Load train/test via Kaggle files (train.csv, test.csv)
- Column dictionary (from data_description.txt)
- Missing values overview and imputation plan
- Numeric distributions and outliers
- Categorical levels and frequency
- Correlations with SalePrice, log-transform check
- Baseline model sanity check


Explanation: We load train/test CSVs and print shapes. Shape reports (rows, columns). Train has target `SalePrice`. We'll use `data_description.txt` for column meanings.


Explanation: dtypes tell us which columns are numeric vs categorical. Missingness guides imputation strategy. Memory helps gauge feasibility for models.


In [13]:
# Setup (auto-download if missing)
import os
import re
import subprocess
import sys
import numpy as np
import pandas as pd
from pathlib import Path

ROOT = Path.cwd()
DATA_DIR = Path('data') / 'raw'
train_path = DATA_DIR / 'train.csv'
test_path = DATA_DIR / 'test.csv'
desc_path = DATA_DIR / 'data_description.txt'

if not train_path.exists() or not test_path.exists():
    proj_kaggle = ROOT / '.kaggle' / 'kaggle.json'
    if proj_kaggle.exists():
        os.environ['KAGGLE_CONFIG_DIR'] = str(proj_kaggle.parent)
    try:
        print('[EDA] Data missing; invoking downloader...')
        subprocess.check_call([sys.executable, str(ROOT / 'scripts' / 'download_kaggle.py')])
    except Exception as e:
        print('[EDA] Download attempt failed:', e)

if not train_path.exists():
    raise FileNotFoundError('Missing data/raw/train.csv even after download attempt. Ensure Kaggle auth and run scripts/download_kaggle.py')

train_df = pd.read_csv(train_path)
if test_path.exists():
    test_df = pd.read_csv(test_path)
else:
    test_df = pd.DataFrame()

print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)
train_df.head()


[EDA] Data missing; invoking downloader...
[EDA] Download attempt failed: Command '['c:\\z_DRIVEs\\ML_MLOps_Projects_2025\\Portfolio_project\\.venv\\Scripts\\python.exe', 'c:\\z_DRIVEs\\ML_MLOps_Projects_2025\\Portfolio_project\\notebooks\\scripts\\download_kaggle.py']' returned non-zero exit status 2.


FileNotFoundError: Missing data/raw/train.csv even after download attempt. Ensure Kaggle auth and run scripts/download_kaggle.py

In [12]:
# Dataset overview: dtypes and memory
info = pd.DataFrame({
    'dtype': train_df.dtypes.astype(str),
    'n_missing': train_df.isna().sum(),
    'pct_missing': (train_df.isna().mean()*100).round(2),
})
print('Total rows:', len(train_df), '| Total columns:', train_df.shape[1])
print('Approx memory (MB):', round(train_df.memory_usage(deep=True).sum() / 1e6, 2))
info.head(20)


NameError: name 'train_df' is not defined

Explanation: Parsed `data_description.txt` to map feature names to meanings. We'll display a sample of the dictionary to help readers understand each column.


Explanation: `SalePrice` is right-skewed; `log1p` often stabilizes variance and improves linear modeling performance.


Explanation: We inspect top numeric correlations. High absolute correlation with `SalePrice` suggests strong linear association; beware multicollinearity among predictors.


Explanation: Cardinality shows how many unique categories per feature. Very high cardinality may require special handling to avoid sparse explosions.


Explanation: Quick baseline with RF + preprocessing shows achievable performance without tuning. Use this as a yardstick for future improvements.


In [ ]:
# Column-wise summary: name, dtype, missing %, description (if available)
cols = []
for col in train_df.columns:
    cols.append({
        'column': col,
        'dtype': str(train_df[col].dtype),
        'pct_missing': float(train_df[col].isna().mean()*100),
        'description': col_desc.get(col, '')
    })
import pandas as pd
col_info = pd.DataFrame(cols).sort_values('column')
col_info.head(25)


In [ ]:
# Visualize top missingness
import matplotlib.pyplot as plt
import seaborn as sns

mis = train_df.isna().mean().sort_values(ascending=False)
mis_top = mis[mis>0].head(20)
plt.figure(figsize=(10,6))
sns.barplot(x=mis_top.values*100, y=mis_top.index, color="#4f8cff")
plt.xlabel('% missing')
plt.ylabel('feature')
plt.title('Top 20 features by missingness')
plt.tight_layout()
plt.show()


Explanation: The bar chart highlights which features need attention for imputation. High-missingness columns may be dropped or encoded carefully depending on importance.


In [ ]:
# Export full column summary for sharing
export_path = Path('artifacts') / 'column_summary.csv'
export_path.parent.mkdir(parents=True, exist_ok=True)
col_info.to_csv(export_path, index=False)
export_path


Explanation: This table can be exported to share with stakeholders; it consolidates column meanings, types, and missingness.


In [ ]:
# Summary statistics
summary = train_df.describe().T
summary


,count,mean,std,min,25%,50%,75%,max
MedInc,20640.0,3.870671,1.899822,0.499900,2.563400,3.534800,4.743250,15.000100
HouseAge,20640.0,28.639486,12.585558,1.000000,18.000000,29.000000,37.000000,52.000000
AveRooms,20640.0,5.429000,2.474173,0.846154,4.440716,5.229129,6.052381,141.909091
AveBedrms,20640.0,1.096675,0.473911,0.333333,1.006079,1.048780,1.099526,34.066667
Population,20640.0,1425.476744,1132.462122,3.000000,787.000000,1166.000000,1725.000000,35682.000000
AveOccup,20640.0,3.070655,10.386050,0.692308,2.429741,2.818116,3.282261,1243.333333
Latitude,20640.0,35.631861,2.135952,32.540000,33.930000,34.260000,37.710000,41.950000
Longitude,20640.0,-119.569704,2.003532,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000
MedHouseVal,20640.0,2.068558,1.153956,0.149990,1.196000,1.797000,2.647250,5.000010
MedHouseValUSD,20640.0,206855.816909,115395.615874,14999.000000,119600.000000,179700.000000,264725.000000,500001.000000


In [ ]:
# Correlations with target
corr = train_df.corr(numeric_only=True)
corr['SalePrice'].sort_values(ascending=False).head(15)


MedHouseValUSD    1.000000
MedHouseVal       1.000000
MedInc            0.688075
AveRooms          0.151948
HouseAge          0.105623
AveOccup         -0.023737
Population       -0.024650
Longitude        -0.045967
AveBedrms        -0.046701
Latitude         -0.144160
Name: MedHouseValUSD, dtype: float64

In [ ]:
# Missing values overview
missing = train_df.isna().mean().sort_values(ascending=False)
missing[missing>0].head(30)


Baseline LinearRegression -> MAE: 53,320 USD | R2: 0.576


In [ ]:
# Column dictionary from data_description.txt (parsed compactly)
text = Path(desc_path).read_text(encoding='utf-8', errors='ignore')
lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
# Heuristic parse: lines like 'MSSubClass: Identifies the type of dwelling involved in the sale.'
col_desc = {}
for ln in lines:
    if ':' in ln and ln.split(':', 1)[0].strip().isidentifier():
        k, v = ln.split(':', 1)
        if len(k) <= 30:
            col_desc[k.strip()] = v.strip()
len(col_desc), list(col_desc.items())[:10]


In [ ]:
# Target distribution (SalePrice) and log-transform check
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(train_df['SalePrice'], kde=True, ax=axes[0])
axes[0].set_title('SalePrice')
sns.histplot(np.log1p(train_df['SalePrice']), kde=True, ax=axes[1])
axes[1].set_title('log1p(SalePrice)')
plt.tight_layout()
plt.show()


In [ ]:
# Numeric features correlation heatmap (top 20 by abs corr with SalePrice)
num_corr = train_df.corr(numeric_only=True)['SalePrice'].abs().sort_values(ascending=False)
top_cols = num_corr.head(20).index
plt.figure(figsize=(10, 8))
sns.heatmap(train_df[top_cols].corr(), cmap='coolwarm', center=0, annot=False)
plt.title('Top numeric correlations')
plt.show()


In [ ]:
# Categorical cardinality and top levels preview
cat_cols = train_df.select_dtypes(exclude=[np.number]).columns.tolist()
card = train_df[cat_cols].nunique().sort_values(ascending=False)
card.head(20)


In [ ]:
# Baseline model: simple pipeline similar to training script
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from app.service.feature_engineering import build_preprocessing_pipeline

# Train/valid split
X = train_df.drop(columns=['SalePrice'])
y = train_df['SalePrice']
pre = build_preprocessing_pipeline(train_df)
model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
from sklearn.pipeline import Pipeline
pipe = Pipeline(steps=[('pre', pre), ('model', model)])
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=42)
pipe.fit(X_tr, y_tr)
preds = pipe.predict(X_va)
print('MAE:', mean_absolute_error(y_va, preds))
print('R2:', r2_score(y_va, preds))
